## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

import os, sys, inspect
currentdir = os.path.dirname(os.path.abspath(inspect.getfile(inspect.currentframe())))
parentdir = os.path.dirname(currentdir)
sys.path.insert(0,parentdir)

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
weather_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
weather_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Pisco,PE,-13.7000,-76.2167,67.01,94,90,8.05,overcast clouds
1,1,Severo-Kurilsk,RU,50.6789,156.1250,38.34,93,83,8.79,broken clouds
2,2,Qaanaaq,GL,77.4840,-69.3632,40.15,91,93,5.06,overcast clouds
3,3,Ushuaia,AR,-54.8000,-68.3000,33.46,93,75,27.63,broken clouds
4,4,Vaini,TO,-21.2000,-175.2000,64.56,100,75,2.30,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 70
What is the maximum temperature you would like for your trip? 95


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & \
                                       (weather_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
5,5,Fare,PF,-16.7000,-151.0167,79.36,72,28,3.36,scattered clouds
7,7,El Alto,PE,-4.2714,-81.2147,74.86,70,36,15.05,scattered clouds
12,12,Half Moon Bay,US,37.4636,-122.4286,71.11,73,90,4.61,overcast clouds
13,13,Rikitea,PF,-23.1203,-134.9692,73.96,71,69,17.11,broken clouds
15,15,San Patricio,US,28.0170,-97.5169,89.69,68,81,10.58,broken clouds
16,16,Padang,ID,-0.9492,100.3543,77.54,89,85,2.24,overcast clouds
19,19,Carutapera,BR,-1.1950,-46.0200,86.70,59,10,6.76,clear sky
21,21,Guatire,VE,10.4762,-66.5427,85.69,86,51,2.48,broken clouds
23,23,Vila Velha,BR,-20.3297,-40.2925,85.96,63,20,8.99,few clouds
27,27,Antalaha,MG,-14.9003,50.2788,72.61,86,1,8.63,clear sky


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                306
City                   306
Country                306
Lat                    306
Lng                    306
Max Temp               306
Humidity               306
Cloudiness             306
Wind Speed             306
Current Description    306
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_clean_df = weather_data_df.loc[(weather_data_df["Max Temp"] <= max_temp) & \
                                       (weather_data_df["Max Temp"] >= min_temp)].dropna()
#preferred_cities_clean_df.head(10)

In [7]:
preferred_cities_clean_df.count()

City_ID                306
City                   306
Country                306
Lat                    306
Lng                    306
Max Temp               306
Humidity               306
Cloudiness             306
Wind Speed             306
Current Description    306
dtype: int64

In [8]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = preferred_cities_clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Fare,PF,79.36,scattered clouds,-16.7000,-151.0167,
7,El Alto,PE,74.86,scattered clouds,-4.2714,-81.2147,
12,Half Moon Bay,US,71.11,overcast clouds,37.4636,-122.4286,
13,Rikitea,PF,73.96,broken clouds,-23.1203,-134.9692,
15,San Patricio,US,89.69,broken clouds,28.0170,-97.5169,
16,Padang,ID,77.54,overcast clouds,-0.9492,100.3543,
19,Carutapera,BR,86.70,clear sky,-1.1950,-46.0200,
21,Guatire,VE,85.69,broken clouds,10.4762,-66.5427,
23,Vila Velha,BR,85.96,few clouds,-20.3297,-40.2925,
27,Antalaha,MG,72.61,clear sky,-14.9003,50.2788,


In [9]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
        
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
   
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found .......skipping.")
        
# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")
      

Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
Hotel not found .......skipping.
-----------------------------
Data Retrieval Complete      
-----------------------------


In [10]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df.head()

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
5,Fare,PF,79.36,scattered clouds,-16.7000,-151.0167,Maitai Lapita Village Huahine
7,El Alto,PE,74.86,scattered clouds,-4.2714,-81.2147,Black Marlin Restaurant & Hospedaje
12,Half Moon Bay,US,71.11,overcast clouds,37.4636,-122.4286,"The Ritz-Carlton, Half Moon Bay"
13,Rikitea,PF,73.96,broken clouds,-23.1203,-134.9692,Pension Maro'i
15,San Patricio,US,89.69,broken clouds,28.0170,-97.5169,"Motel 6 Sinton, TX"


In [11]:
# 7b. Determine if there are any empty rows and drop the rows where there is no Hotel Name
hotel_df.count()

City                   306
Country                306
Max Temp               306
Current Description    306
Lat                    306
Lng                    306
Hotel Name             306
dtype: int64

In [12]:
# 7c. Create new DataFrame of Hotel Name.)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.count()

City                   306
Country                306
Max Temp               306
Current Description    306
Lat                    306
Lng                    306
Hotel Name             306
dtype: int64

In [13]:
# 8a. Create the output File (CSV)
output_data_file = "../Vacation_Search/WeatherPy_vacation.csv"

# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [14]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""
# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [15]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=2.0)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,
             dissipating=False, max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
#fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))